In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from typing import Tuple

In [ ]:
# ===============================================
# DATA GENERATION CLASS (for transparency)
# ===============================================
# This class shows how our training data was generated.
# The actual data used in this experiment is loaded from a pickle file below.

class AdjacencyMatrixGenerator:
    """
    A class to generate adjacency matrices for directed and undirected graphs.
    This code is included for transparency - actual data is loaded from pickle file.
    """

    def __init__(self):
        self.matrices = []
        self.labels = []

    def generate_symmetric_matrix(self, n: int) -> np.ndarray:
        """Generate a symmetric adjacency matrix (undirected graph)."""
        matrix = np.random.randint(0, 2, size=(n, n))
        matrix = np.triu(matrix) + np.triu(matrix, 1).T
        np.fill_diagonal(matrix, 0)
        return matrix

    def generate_non_symmetric_matrix(self, n: int) -> np.ndarray:
        """Generate a non-symmetric adjacency matrix (directed graph)."""
        matrix = np.random.randint(0, 2, size=(n, n))
        np.fill_diagonal(matrix, 0)
        if np.array_equal(matrix, matrix.T):
            for i in range(n):
                for j in range(i+1, n):
                    if matrix[i, j] == matrix[j, i]:
                        matrix[i, j] = 1 - matrix[i, j]
                        break
                else:
                    continue
                break
        return matrix

    def generate_dataset(self, n: int, num_symmetric: int, num_non_symmetric: int):
        """Generate a dataset of adjacency matrices."""
        self.matrices = []
        self.labels = []

        for _ in range(num_symmetric):
            matrix = self.generate_symmetric_matrix(n)
            self.matrices.append(matrix)
            self.labels.append(0)

        for _ in range(num_non_symmetric):
            matrix = self.generate_non_symmetric_matrix(n)
            self.matrices.append(matrix)
            self.labels.append(1)

    def save_to_pickle(self, filename: str):
        """Save the generated matrices and labels to a pickle file."""
        data = {
            'matrices': self.matrices,
            'labels': self.labels,
            'matrix_size': len(self.matrices[0]) if self.matrices else 0
        }

        with open(filename, 'wb') as f:
            pickle.dump(data, f)

        print(f"Saved {len(self.matrices)} matrices to {filename}")

    def load_from_pickle(self, filename: str):
        """Load matrices and labels from a pickle file."""
        with open(filename, 'rb') as f:
            data = pickle.load(f)

        self.matrices = data['matrices']
        self.labels = data['labels']

        print(f"Loaded {len(self.matrices)} matrices from {filename}")

    def get_flattened_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """Get matrices flattened into vectors for neural network input."""
        if not self.matrices:
            return np.array([]), np.array([])

        flattened_matrices = np.array([matrix.flatten() for matrix in self.matrices])
        labels_array = np.array(self.labels)

        return flattened_matrices, labels_array


In [ ]:
# Luis Hernandez, Roberto Palacios
# Note: this code was partially generated by AI (Grok 3)

# Sigmoid activation function to map values to (0, 1)
def sigmoid(x):
    # Compute 1 / (1 + e^-x) element-wise
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid for backpropagation, note that this takes in activated output and not raw input
def sigmoid_derivative_from_output(x):
    # Compute sigmoid(x) * (1 - sigmoid(x)) for the gradient
    return x * (1 - x)

# Neural Network class to classify graphs
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initializing weights and biases with small random values
        # input_size = 16 (4x4 flattened), hidden_size = number of hidden neurons, output_size = 1
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01  # Input to hidden weights
        self.b1 = np.zeros((1, hidden_size))  # Hidden layer biases
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01  # Hidden to output weights
        self.b2 = np.zeros((1, output_size))  # Output biases
    # Perform forward propagation through the network
    def forward(self, X):
        # Compute hidden layer input: X * W1 + b1
        self.z1 = np.dot(X, self.W1) + self.b1
        # Apply sigmoid activation to hidden layer
        self.a1 = sigmoid(self.z1)
        # Compute output layer input: a1 * W2 + b2
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        # Apply sigmoid activation to output layer
        self.a2 = sigmoid(self.z2)
        # Return final output
        return self.a2

    # Backward propagation to update weights & biases
    def backward(self, X, y, output, learning_rate):
        # Compute error at output layer, true means predicted
        self.error = y - output
        # Compute output layer gradient, error * sigmoid derivative
        self.delta2 = self.error * sigmoid_derivative_from_output(output)

        # Propagate error to hidden layer: delta2 * W2^T
        self.error_hidden = np.dot(self.delta2, self.W2.T)
        # Compute hidden layer gradient: error_hidden * sigmoid derivative
        self.delta1 = self.error_hidden * sigmoid_derivative_from_output(self.a1)

        # Update weights & biases using gradient descent
        self.W2 += learning_rate * np.dot(self.a1.T, self.delta2)  # Update hidden to output weights
        self.b2 += learning_rate * np.sum(self.delta2, axis=0, keepdims=True)  # Update output biases
        self.W1 += learning_rate * np.dot(X.T, self.delta1)  # Update input to hidden weights
        self.b1 += learning_rate * np.sum(self.delta1, axis=0, keepdims=True)  # Update hidden biases

    # Train the network for the specified number of epochs
    def train(self, X, y, epochs, learning_rate):
        errors = []  # List to store mean squared error per epoch
        for epoch in range(epochs):
            # Forward pass to get predictions
            output = self.forward(X)
            # Backward pass to update parameters
            self.backward(X, y, output, learning_rate)
            # Compute mean squared error for this epoch
            mse = np.mean(np.square(y - output))
            errors.append(mse)
            # Print progress every 1000 epochs
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, MSE: {mse:.6f}")
        # Return list of errors for plotting
        return errors

In [ ]:
# Hyperparameters for neural network
input_size = 16  # 4x4 matrix flattened into a 16-element vector
hidden_size = 32  # Number of neurons in the hidden layer, adjustable
output_size = 1   # Single output neuron: 0 for undirected, 1 for directed
learning_rate = 0.01  # Step size for gradient descent
epochs = 10000  # Number of training iterations
num_samples = 10  # Total number of graph samples (5 directed, 5 undirected)

# ===============================================
# LOAD TRAINING DATA
# ===============================================
# Load data using our class methods
generator = AdjacencyMatrixGenerator()

# Option 1: Load from pickle file (recommended for reproducibility)
try:
    generator.load_from_pickle("neural_network_training_data.pkl")
    print("Using pre-generated dataset for reproducible results")
except FileNotFoundError:
    print("Pickle file not found. Generating new dataset...")
    generator.generate_dataset(n=4, num_symmetric=num_samples//2, num_non_symmetric=num_samples//2)
    generator.save_to_pickle("neural_network_training_data.pkl")

# Get data ready for neural network
X, y = generator.get_flattened_data()
y = y.reshape(-1, 1)  # Reshape for neural network compatibility

print(f"Dataset ready: {len(X)} samples, {X.shape[1]} features each")

# Initialize the neural network with specified architecture
nn = NeuralNetwork(input_size, hidden_size, output_size)
# Train the network and collect error history
errors = nn.train(X, y, epochs, learning_rate)

# Plot the mean squared error over training epochs
plt.plot(errors)  # Plot error values
plt.xlabel('Epoch')  # Label x-axis
plt.ylabel('Mean Squared Error')  # Label y-axis
plt.title('Training Error vs Epoch')  # Set plot title
plt.grid(True)  # Add grid for better readability
plt.show()  # Display the plot

# Test the trained network on the dataset
print("\nTesting network:")
for i in range(num_samples):
    # Get prediction for each sample
    prediction = nn.forward(X[i:i+1])[0][0]
    # Get true label
    actual = y[i][0]
    # Print predicted and actual values
    print(f"Sample {i+1}: Predicted: {prediction:.3f}, Actual: {actual}")